In [2]:
from crawler import craw
import datetime
import os 
import json 

current_semester = craw.fetch_semester()
output_path = f"../api/{current_semester}"

initial_fetch = False

if not os.path.exists(output_path):
    initial_fetch = True
    os.makedirs(output_path)

colleges,departments = craw.fetch_departments()
json.dump(colleges,    open(f"{output_path}/colleges.json",    "w"))
json.dump(departments, open(f"{output_path}/departments.json", "w"))
all_courses = []

for index in range(len(departments)):
    dep  = departments[index]
    courses = craw.fetch_courses(dep)
    all_courses += courses
json.dump(courses, open(f"{output_path}/all_course.json", "w"))

failed = []
for course in all_courses:
    initial_fetch = False
    try:
        course_detail = craw.fetch_course_detail(course)
    except Exception as e:
        print(f"[Fetal Error] Failed to fetch course detail for serial:{course['serial']}")
        failed.append(course)
        continue
    
    if not os.path.exists(f"{output_path}/{course['serial']}"):
        os.makedirs(f"{output_path}/{course['serial']}")
        os.makedirs(f"{output_path}/{course['serial']}/history")
        initial_fetch = True
        temp = {'timestamp':[datetime.datetime.now().timestamp()], 
                'data'     :{'selected':course_detail['selected'],
                             'assigned':course_detail['assigned'],
                             'preselecStu':course_detail['preselecStu'],
                             'stuGender':course_detail['stuGender'],}
        }
        json.dump(temp, open(f"{output_path}/{course['serial']}/history/statistics.json", "w"))

    if not initial_fetch:
        last_statistics = json.load(open(f"{output_path}/{course['serial']}/history/statistics.json"))
        last_statistics['timestamp'].append(datetime.datetime.now().timestamp())
        last_statistics['data']['selected'].append(course_detail['selected'])
        last_statistics['data']['assigned'].append(course_detail['assigned'])
        last_statistics['data']['preselecStu'].append(course_detail['preselecStu'])
        last_statistics['data']['stuGender'].append(course_detail['stuGender'])
        json.dump(course_detail, open(f"{output_path}/{course['serial']}/history/statistics.json", "w"))
    json.dump(course_detail, open(f"{output_path}/{course['serial']}/detial.json", "w"))
    
    
    # stu_list history name (current date)
    his_name = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
    json.dump(course_detail['stus'], open(f"{output_path}/{course['serial']}/history/{his_name}.json", "w"))
status = {'update_time':datetime.datetime.now().strftime('%Y-%m-%d-%H-%M'),
          'total_colleges'   :len(colleges),
          'total_departments':len(departments), 
          'total_courses'    :len(all_courses),
          'failed'           :len(failed)}
json.dump(status, open(f"{output_path}/status.json", "w"))
json.dump(failed, open(f"{output_path}/failed.json", "w"))
    

[Work] Fetching current semester ......
[Done] Current semester: 1122
[Work] Fetching departments ......
Parsing Chinese department data ......
Parsing Chinese department data ......
[Done] All departments fetched ......
[Work] Fetching departments ......
Target department: Center of Teacher-training Program
Found 1 pages of courses ......
Parsing Chinese course data ......
Parsing English course data ......
[Done] All departments fetched ......
[Work] Fetching departments ......
Target department: Department of Chinese Literature
Found 3 pages of courses ......
Parsing Chinese course data ......
Parsing English course data ......
[Done] All departments fetched ......
[Work] Fetching departments ......
Target department: Department of English
Found 1 pages of courses ......
Parsing Chinese course data ......
Parsing English course data ......
[Done] All departments fetched ......
[Work] Fetching departments ......
Target department: Department of French
Found 1 pages of courses ......


KeyError: 'timestamp'